In [82]:
from scipy.stats import f_oneway
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib
import matplotlib.pyplot as plt
from itertools import combinations
import statistics 

## DATA

In [106]:
# data = pd.read_csv("test_data/test_data_0406.csv")
data = pd.read_csv("test_data/test_data.csv")
data['ratio'] = data['set'].apply(lambda x: x.split('_')[3])
data['schedule_size'] = data['set'].apply(lambda x: x.split('_')[1])

data.reward = data.reward.round(6)
data['acceptable']= [1 if i == 1 else 0 for i in data['reward']]

conditions = [
            data['schedule_size'] =='easy',
            data['schedule_size']=='medium',
            data['schedule_size']=='hard',
            data['schedule_size']=='extrahard',
            data['schedule_size']=='xxhard',
        ]

outputs = ['ms8','ms14','ms18','ms23','ms30']
X = np.select(conditions, outputs, 'Other')
data['Problem Complexity (max shifts)'] = X

data.shape


(20000, 9)

In [33]:
data.head(1)

,problem,reward,model,seed,set,ratio,schedule_size,acceptable,Problem Complexity (max shifts)
0,101,1.0,step_8,10,shifts_easy_ratio_mixed,mixed,easy,1,08


## TESTING

In [132]:
def welch_test(data1, data2, alpha=0.05, tail=2):
    """
    Wraps around ttest_ind function of scipy, without assuming equal variances.
    Params
    ------
    - data1 (ndarray of dim 1)
    The performance measures of Algo1.
    - data2 (ndarray of dim 1)
    The performance measures of Algo2.
    - alpha (float in ]0,1[)
    The significance level used by the Welch's t-test.
    - tail (1 or 2)
    Perform a one tail or two tail test.
    """

    assert tail==1 or tail==2, "tail should be one or two, referring to the one-sided or two-sided t-test."
    data1 = data1.squeeze()
    data2 = data2.squeeze()
    assert alpha <1 and alpha >0, "alpha should be between 0 and 1"

    t, p = stats.ttest_ind(data1, data2, equal_var=False)

    if tail==1:
        alpha = 2*alpha
    if p <= alpha:
        if t<0:
            print("Result of the Welch's t-test at level %02g: μ2>μ1, the test passed with p-value = %02g." %(alpha, p))
            print(f'd2 (m={statistics.mean(data2)}, sd={statistics.stdev(data2)}), d2 (m={statistics.mean(data1)}, sd={statistics.mean(data1)}), {alpha}, {p}')
            
            return alpha, p
        else:
            print("Result of the Welch's t-test level %02g: μ1>μ2, the test passed with p-value = %02g." %(alpha, p))
            print(f'd1 (m={statistics.mean(data1)}, sd={statistics.stdev(data1)}), d2 (m={statistics.mean(data2)}, sd={statistics.mean(data2)}), {alpha}, {p}')
            
            return alpha, p
    else:
        print("Results of the Welch's t-test level %02g: there is not enough evidence to prove any order relation between μ1 and μ2." % alpha)
        return alpha, 0
    
    print("Welch's t-test done.")

In [48]:
# reward comparison
data[data['model']=='terminal_8']
data[data['model']=='step_8']
data[data['model']=='stepbonus_8']
data[data['model']=='random_0']

# schedule_size comparison
ms8 = data['reward'][data['Problem Complexity (max shifts)']=='ms8'] 
ms14 = data['reward'][data['Problem Complexity (max shifts)']=='ms14']
ms18 = data['reward'][data['Problem Complexity (max shifts)']=='ms18']
ms23 = data['reward'][data['Problem Complexity (max shifts)']=='ms23']
ms30 = data['reward'][data['Problem Complexity (max shifts)']=='ms30']

# ratio comparison
average_ratio = data['reward'][data['ratio']=='mixed'] 
high_ratio = data['reward'][data['ratio']=='above'] 

average_ratio_acceptable = data['acceptable'][data['ratio']=='mixed'] 
high_ratio_acceptable = data['acceptable'][data['ratio']=='above'] 

In [84]:
data1 = average_ratio_acceptable.to_numpy()
data2 = high_ratio_acceptable.to_numpy()
data1 = average_ratio.to_numpy()
data2 = high_ratio.to_numpy()
data1 = data['reward'][data['model']=='stepbonus_8'].to_numpy()
data2 = data['reward'][data['model']=='random_0'].to_numpy()

In [ ]:
# model

combs = [(i,j) for i, j in combinations(data['model'].unique().tolist(), 2)]

terminal_8 = data['reward'][data['model']=='terminal_8'].to_numpy()
step_8 = data['reward'][data['model']=='step_8'].to_numpy()
stepbonus_8 = data['reward'][data['model']=='stepbonus_8'].to_numpy()
random_0 = data['reward'][data['model']=='random_0'].to_numpy()

for i in combs:
    print(i)
    welch_test(eval(i[0]),eval(i[1]))




In [123]:
# ratio

combs = [(i,j) for i, j in combinations(data['ratio'].unique().tolist(), 2)]

average_ratio = data['reward'][(data['ratio']=='mixed') & (data['model']!='random_0') & (data['model']=='stepbonus_8')].to_numpy()
high_ratio = data['reward'][(data['ratio']=='above') & (data['model']!='random_0') & (data['model']=='stepbonus_8')].to_numpy()

welch_test(average_ratio,high_ratio)


Result of the Welch's t-test level 0.05: μ1>μ2, the test passed with p-value = 5.71597e-120.
d1 (m=0.9591988016, sd=0.14820917206815876), d2 (m=0.8140200936, sd=0.8140200936)


5.715967001040617e-120

In [ ]:
# max shifts

mdata = data[data['model']!='random_0']

combs = [(i,j) for i, j in combinations(data['Problem Complexity (max shifts)'].unique().tolist(), 2)]


ms8 = mdata['reward'][mdata['Problem Complexity (max shifts)']=='ms8'] 
ms14 = mdata['reward'][mdata['Problem Complexity (max shifts)']=='ms14']
ms18 = mdata['reward'][mdata['Problem Complexity (max shifts)']=='ms18']
ms23 = mdata['reward'][mdata['Problem Complexity (max shifts)']=='ms23']
ms30 = mdata['reward'][mdata['Problem Complexity (max shifts)']=='ms30']

for i in combs:
    print(i)
    welch_test(eval(i[0]),eval(i[1]))
    print('\n')

In [ ]:

mdata = data[(data['model']!='random_0') & (data['ratio']!='high')]

combs = [(i,j) for i, j in combinations(data['Problem Complexity (max shifts)'].unique().tolist(), 2)]


ms8 = mdata['reward'][mdata['Problem Complexity (max shifts)']=='ms8'] 
ms14 = mdata['reward'][mdata['Problem Complexity (max shifts)']=='ms14']
ms18 = mdata['reward'][mdata['Problem Complexity (max shifts)']=='ms18']
ms23 = mdata['reward'][mdata['Problem Complexity (max shifts)']=='ms23']
ms30 = mdata['reward'][mdata['Problem Complexity (max shifts)']=='ms30']

for i in combs:
    print(i)
    welch_test(eval(i[0]),eval(i[1]))
    print('\n')

In [147]:

ndata = data[(data['model']=='stepbonus_8') & (data['Problem Complexity (max shifts)']!='ms30')  & (data['ratio']!='high')]


ndata.reward.mean()

0.8710594505

In [148]:
sum(ndata.acceptable / len(ndata))

0.769499999999943

In [149]:

mdata = data[data['model']=='stepbonus_8']

ms8 = mdata['reward'][(mdata['Problem Complexity (max shifts)']=='ms8') & (mdata['ratio']=='average')] 
ms30 = mdata['reward'][(mdata['Problem Complexity (max shifts)']=='ms8') & (mdata['ratio']=='high')]

welch_test(ms8,ms30)

Results of the Welch's t-test level 0.05: there is not enough evidence to prove any order relation between μ1 and μ2.


(0.05, 0)